# b-29

In [1]:
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.saving import register_keras_serializable

2025-04-17 20:40:01.801460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744909801.814870   23510 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744909801.819344   23510 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-17 20:40:01.832873: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
california_housing = fetch_california_housing()
california_housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]], shape=(20640, 8)),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': 

# Splitting data into train and test.

In [3]:
X_train0, X_test, Y_train0, Y_test  = train_test_split(california_housing["data"], california_housing["target"])

In [4]:
X_train1, X_validation, Y_train1, Y_validation = train_test_split(X_train0, Y_train0)

In [5]:
X_test.shape

(5160, 8)

In [6]:
X_train0.shape

(15480, 8)

In [7]:
X_train1.shape

(11610, 8)

In [8]:
X_validation.shape

(3870, 8)

# Normalizing data
using "sklearn.preprocessing.StandardScaler" function for normalizing data

In [9]:
sc = StandardScaler()
X_train_s = sc.fit_transform(X_train1)
X_validation_s = sc.fit_transform(X_validation)
X_test_s = sc.transform(X_test)

--------------------------------

In [10]:
X_train0.shape[1:]

(8,)

In [11]:
X_train_s_1, X_train_s_2 = X_train_s[:, :6], X_train_s[:, :-4]
X_validation_s_1, X_validation_s_2 = X_validation_s[:, :6], X_validation_s[:, :-4]
X_test_s_1, X_test_s_2 = X_test_s[:, :6], X_test_s[:, :-4]

# Define Model in Subclass API

In [12]:
@register_keras_serializable()
class WideAndDeepANN(keras.Model):
    def __init__(self, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden_layer_1 = keras.layers.Dense(50, activation=activation)
        self.hidden_layer_2 = keras.layers.Dense(10, activation=activation)
        self.final_output = keras.layers.Dense(1)
        self.helper_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_1, input_2 = inputs
        h1_out = self.hidden_layer_1(input_1)
        h2_out = self.hidden_layer_2(h1_out)
        concat_out = keras.layers.concatenate([input_2, h2_out])
        final_out = self.final_output(concat_out)
        helper_out = self.helper_output(h2_out)
        return final_out, helper_out

In [13]:
model_f = WideAndDeepANN()

# Compile Model

In [14]:
model_f.compile(loss = ["mse", "mse"],
              loss_weights = [0.8, 0.2],
              optimizer = "sgd",
              metrics = [["mean_absolute_error"], ["mean_absolute_error"]])

I0000 00:00:1744909804.872162   23510 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2229 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5


# Fit Model

In [15]:
model_f.fit((X_train_s_1, X_train_s_2),(Y_train1, Y_train1), epochs=30, validation_data = ((X_validation_s_1, X_validation_s_2), (Y_validation, Y_validation)))

Epoch 1/30


I0000 00:00:1744909806.120761   23556 service.cc:148] XLA service 0x7cb674003ff0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744909806.120780   23556 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 SUPER, Compute Capability 7.5
2025-04-17 20:40:06.140984: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1744909806.184794   23556 cuda_dnn.cc:529] Loaded cuDNN version 90800


 66/363 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.2931 - mean_absolute_error: 1.4151 - mean_absolute_error_1: 1.7558 - mse_loss: 5.1346

I0000 00:00:1744909806.765468   23556 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


363/363 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 2.1153 - mean_absolute_error: 0.8949 - mean_absolute_error_1: 1.1103 - mse_loss: 2.6788 - val_loss: 0.6010 - val_mean_absolute_error: 0.5499 - val_mean_absolute_error_1: 0.5995 - val_mse_loss: 0.6626
Epoch 2/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5885 - mean_absolute_error: 0.5546 - mean_absolute_error_1: 0.5970 - mse_loss: 0.6493 - val_loss: 0.5683 - val_mean_absolute_error: 0.5421 - val_mean_absolute_error_1: 0.5721 - val_mse_loss: 0.6091
Epoch 3/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5654 - mean_absolute_error: 0.5439 - mean_absolute_error_1: 0.5750 - mse_loss: 0.6134 - val_loss: 0.5257 - val_mean_absolute_error: 0.5149 - val_mean_absolute_error_1: 0.5425 - val_mse_loss: 0.5695
Epoch 4/30
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5228 - mean_absolute_error: 0.5244 - mean_absolute_error_1: 0.5472 - mse_loss: 0.5572 - val_loss: 0.5079 - val_mean_absolute_error: 0.5060 - val_mean_absolute_error_1: 0

# Save Model

In [16]:
model_f.save("housing_reg_model.keras")

In [17]:
model_f_reg = keras.models.load_model("housing_reg_model.keras")

In [18]:
model_f_reg.summary()

Model: "wide_and_deep_ann"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 50)             │           350 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │           510 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            15 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 888 (3.47 KB)

 Trainable params: 886 (3.46 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)